In [ ]:
import requests
parameter = {
    'id': 'ethereum',
    'vs_currency': 'usd',
    'from': '1650982222',
    'to': '1658758222'
}
response = requests.get('https://api.coingecko.com/api/v3/coins/ethereum/market_chart/range?vs_currency=usd&from=1650982222&to=1658758222')

print(len(response.json()['prices']))

In [ ]:
prices = [tpl[1] for tpl in response.json()['prices']]

print(prices)

In [ ]:
import mesa
from decimal import *
import pandas as pd
from IPython.display import display

from contracts import router_factory
from agents.buyer import BuyerAgent
from agents.oracle import Oracle
from states.params import Params
from contracts.types import DummyProtocolAgent, Tokens

"""Model Data Collector Methods"""

def eth_prices(model):
    return Oracle.get_price_of("ETH")

def sa_balance(model):
    return model.router._sa_pool.balance


def fee_balance(model):
    return model.router._fee_pool.balance


def va_balance(model):
    return model.router._va_pool.balance * Oracle.get_price_of("ETH")

def principal(model):
    return model.router._sa_pool.principal


def total(model):
    return (
        model.router._sa_pool.balance
        + model.router._fee_pool.balance
        + model.router._va_pool.balance * Oracle.get_price_of("ETH")
    )


def num_triggered(model):
    return model.router.num_triggered


def num_rebalanced(model):
    return model.router.num_rebalanced


class LifelyPayModel(mesa.Model):
    def __init__(self, n):
        super().__init__()
        self._step = 0
        self.router = router_factory.Router("ETH", "USDC")

        # Initiate w/ $1M Protocol-injected Liquidity
        self.router.process_lp_provider_request(
            DummyProtocolAgent(), Tokens(Decimal(3000000), "USDC")
        )

        self.schedule = mesa.time.RandomActivation(self)
        for i in range(n):
            ba = BuyerAgent(i, "DIMWIT-" + str(i), self.router, self)
            # buyer gets infinite ETH to spend
            # amount paid and amount redeemed is tracked separately
            ba.initiate_with("ETH")
            self.schedule.add(ba)

        self.running = True
        self.datacollector = mesa.DataCollector(
            model_reporters={
                "ETH Prices": eth_prices,
                "USDC Pool Balance USD": sa_balance,
                "ETH Pool Balance USD": va_balance,
                "Fee Pool Balance USDC": fee_balance,
                "Total Asset Value USD": total,
                "# Emergency Triggers": num_triggered,
                "# Pool Rebalancing": num_rebalanced,
            },
            agent_reporters={
                "buyer_spent_eth_usd": "spent_eth_usd",
                "buyer_redeemed_eth_usd": "redeemed_eth_usd",
                "buyer_remaining_vouchers": "remaining_vouchers",
            },
        )

    def step(self):
        Oracle.set_price(Decimal(str(prices[self._step])))
        self._step += 1
        self.schedule.step()
        self.datacollector.collect(self)


if __name__ == "__main__":
    getcontext().prec = 18
    results = mesa.batch_run(
        LifelyPayModel,
        parameters={"n": 50},
        iterations=1,
        max_steps=500,
        number_processes=1,
        data_collection_period=1,
        display_progress=False,
    )
    Params.hard_reset()
    Oracle.reset()
    rdf = pd.DataFrame(results)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
pd.set_option('display.max_rows', None)
one_iteration = rdf[(rdf.iteration == 0)]

filtered = one_iteration.loc[(one_iteration['AgentID'] == 0)]

filtered['Total Asset Value USD'] = filtered['Total Asset Value USD'].astype(float)

filtered['ETH Prices'] = filtered['ETH Prices'].astype(float)
filtered['ETH Prices'] = filtered['ETH Prices'].mul(1.5e3)


ax = plt.gca()
ax.set_yscale('log')

filtered.plot(kind='line',x='Step',y='Total Asset Value USD', ax=ax)
filtered.plot(kind='line',x='Step',color='green',y='ETH Prices', ax=ax)

valid_df = one_iteration.loc[(one_iteration['AgentID'] == 0)] #(one_iteration['buyer_spent_eth'] != 0)]

display(valid_df)